<a href="https://colab.research.google.com/github/marquezjorge/456-Camo/blob/main/lenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing the libraries

In [ ]:
import tensorflow as tf
import numpy as np
import os
import sklearn
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import keras
from google.colab import drive


In [ ]:
drive.mount('/content/gdrive')
!unzip gdrive/My\ Drive/CECS456/camo3.zip

In [ ]:
x = [] # list for data
y = [] # list for labels
rfoldername = "camo3" # root foldername
cfoldername = ["Background", "Camouflaged", "Non-camouflaged"] # class foldernames
# loop for integer encoding 
for i in range(0,3):
  # iterate through each image in each folder
  for imgname in os.listdir(rfoldername+'/'+cfoldername[i]):
    # read image from a file into array
    img = mpimg.imread(rfoldername+'/'+cfoldername[i]+'/'+imgname)
    # shrinking the image tolenet input size
    rsized_img = cv2.resize(img, (32, 32), interpolation=cv2.INTER_AREA)
    # matrix of all images
    x.append(rsized_img) 
    # appending label number to list 
    y.append(i) # 0 - Background, 1 - Camouflaged, 2 - Non-camouflaged  
# change data and labels to numpy arrays
x = np.array(x)
y = np.array(y)

In [ ]:
# splitting dataset into the training set, validation set and test set
# all data split 70:15:15
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.15, random_state = 26)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train,test_size=0.20, random_state= 26)

In [ ]:
# coverting labels from integer encoding to one-hot encoding
y_train = to_categorical(y_train, dtype ="uint8")
y_test = to_categorical(y_test, dtype ="uint8")
y_valid = to_categorical(y_valid, dtype ="uint8")

In [ ]:
# normalize data
x_train = x_train /255
x_test = x_test /255
x_valid = x_valid /255

In [ ]:
# building the model
model = keras.Sequential([
    # 3 convolutional layers
    # 2 subsampling layers
    # 2 dropout layers
    keras.layers.Conv2D(filters=6, kernel_size=5, activation='relu', padding='same', input_shape=(32, 32, 3)),
    
    keras.layers.MaxPool2D(pool_size=2, strides=2),
    keras.layers.Dropout(.2, input_shape=(2,)),

    keras.layers.Conv2D(filters=64, kernel_size=5,activation='relu'),
   
    keras.layers.MaxPool2D(pool_size=2, strides=2),
    keras.layers.Dropout(.2, input_shape=(2,)),

    keras.layers.Conv2D(filters=128, kernel_size=5,activation='relu'),

    keras.layers.Flatten(),
    # 2 fully connected layers
    keras.layers.Dense(72, activation='relu'), 
    keras.layers.Dense(36, activation='relu'),
    # output layer 
    keras.layers.Dense(3, activation='softmax')
])

In [ ]:
# compiling the model
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
               metrics=["accuracy"])

In [ ]:
# training the model
history = model.fit(x_train, y_train, batch_size= 16, 
          epochs=10, validation_data=(x_valid, y_valid))

In [ ]:
# evaluation
test_accuracy = model.evaluate(x_test, y_test)
print('Total Loss:', test_accuracy[0])
print('Accuracy Testing:', test_accuracy[1])

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()